In [241]:
import numpy
import cPickle as pickle
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils
from keras.optimizers import RMSprop
numpy.random.seed(260389)

In [242]:
data = numpy.loadtxt(open('profiles_trimmed','rb'))
annotation = numpy.loadtxt(open('numpy_annotations','rb'))

In [243]:
cnv_dict = {}

In [244]:
#function for padding sequences
def pad_seq(seq, max_length):
    temp = numpy.zeros(max_length)
    for i in xrange(len(seq)):
        temp[i] = seq[i]
    return temp

#creating the data dictionnary
for i in xrange(data.shape[0]):
    if not ((data[i,0],data[i,1]) in cnv_dict.keys()):
        cnv_dict[(data[i,0],data[i,1])] = [data[i,3]]
    else: cnv_dict[(data[i,0],data[i,1])].append(data[i,3])

In [245]:
#finding the max length of sequence in the dictionnary
m = [len(cnv_dict[i]) for i in cnv_dict.keys()]
max_length = numpy.max(m)

In [246]:
#padding the sequences
for i in cnv_dict.keys():
    cnv_dict[i] = pad_seq(cnv_dict[i], max_length)

In [247]:
pickle.dump(cnv_dict,open("cnv_dict.p","wb"))

In [248]:
#making the annotation dictionnary
annot_dict = {}
for i in xrange(annotation.shape[0]):
    annot_dict[(annotation[i,0],annotation[i,1])] = [annotation[i,2]]


In [249]:
#creating a dataset palatable for the neural network (a numpy array...)
Xdata = numpy.zeros((len(cnv_dict.keys()), max_length))
Ydata = numpy.zeros((len(cnv_dict.keys())))
Xtest = numpy.zeros((len(cnv_dict.keys()), max_length))
Ytest = numpy.zeros((len(cnv_dict.keys())))


unannotated = [] # we are keeping the unannotated sequences to see if we can possibly predict them later on.

In [250]:
#creating the dataset for the neural network (and removing a chromosome for testing purposes (cross-validation))
chrom=11.

'''
TODO: check out the bug with the indices!
'''
for i in xrange(len(cnv_dict.keys())):
    key = cnv_dict.keys()[i]
    if key in annot_dict.keys():
        if key[1]==chrom:
            Xtest[i,:] = cnv_dict[key]
            Ytest[i] = annot_dict[key][0]            
        else:
            Xdata[i,:] = cnv_dict[key]
            Ydata[i] = annot_dict[key][0]
    else:
        unannotated.append(key)

In [251]:
#scaling data (or not...)
#Xdata = scale(Xdata)

In [252]:
Xdata = Xdata.reshape(Xdata.shape + (1,))
Ydata = np_utils.to_categorical(Ydata,nb_classes=2)
Xtest = Xtest.reshape(Xtest.shape + (1,))
Ytest = np_utils.to_categorical(Ytest,nb_classes=2)

numpy.save('padded_seq',Xdata)
numpy.save('data_annotation',Ydata)

In [255]:
###constructing the neural network (keras)
model = Sequential()
model.add(Convolution1D(32,3, input_shape=(max_length,1)))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(2))
opt = RMSprop(lr=1e-4)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [257]:
model.fit(Xdata,Ydata,verbose=1,batch_size=200, validation_split = 0.2, nb_epoch=25)

Train on 2760 samples, validate on 690 samples
Epoch 1/25
2760/2760 [==============================] - 31s - loss: 1.2800 - acc: 0.8337 - val_loss: 1.1492 - val_acc: 0.7986
Epoch 2/25
2760/2760 [==============================] - 25s - loss: 1.1912 - acc: 0.6152 - val_loss: 0.6421 - val_acc: 0.3116
Epoch 3/25
2760/2760 [==============================] - 24s - loss: 0.7152 - acc: 0.4152 - val_loss: 0.8218 - val_acc: 0.7652
Epoch 4/25
2760/2760 [==============================] - 25s - loss: 0.5629 - acc: 0.6243 - val_loss: 0.5067 - val_acc: 0.4333
Epoch 5/25
2760/2760 [==============================] - 25s - loss: 0.4932 - acc: 0.6174 - val_loss: 0.4923 - val_acc: 0.3551
Epoch 6/25
2760/2760 [==============================] - 25s - loss: 0.4557 - acc: 0.5482 - val_loss: 0.5578 - val_acc: 0.3522
Epoch 7/25
2760/2760 [==============================] - 25s - loss: 0.4799 - acc: 0.4428 - val_loss: 0.4769 - val_acc: 0.6986
Epoch 8/25
2760/2760 [==============================] - 25s - loss: 0.5

In [179]:
numpy.save('padded_seq',Xdata)
numpy.save('data_annotation',Ydata)

In [192]:
numpy.min(Xdata)

-7.6749999999999998

In [193]:
from sklearn.preprocessing import scale

In [233]:
unannotated

[(247.0, 1.0),
 (306.0, 17.0),
 (118.0, 2.0),
 (441.0, 11.0),
 (267.0, 17.0),
 (181.0, 1.0),
 (419.0, 17.0),
 (290.0, 17.0),
 (281.0, 17.0),
 (368.0, 1.0),
 (147.0, 1.0),
 (264.0, 1.0),
 (287.0, 4.0),
 (182.0, 4.0),
 (207.0, 4.0),
 (161.0, 11.0),
 (375.0, 11.0),
 (14.0, 1.0),
 (79.0, 1.0),
 (211.0, 17.0),
 (57.0, 17.0),
 (287.0, 17.0),
 (302.0, 4.0),
 (133.0, 17.0),
 (188.0, 17.0),
 (25.0, 17.0),
 (368.0, 3.0),
 (38.0, 11.0),
 (312.0, 1.0),
 (368.0, 4.0),
 (495.0, 17.0),
 (487.0, 4.0)]

In [218]:
numpy.max(Xdata)

57.231199909691945

In [224]:
model.layers[1].output_shape

(None, 2967, 32)

In [230]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_28 (Convolution1D) (None, 5935, 32)      128         convolution1d_input_26[0][0]     
____________________________________________________________________________________________________
maxpooling1d_24 (MaxPooling1D)   (None, 2967, 32)      0           convolution1d_28[0][0]           
____________________________________________________________________________________________________
flatten_22 (Flatten)             (None, 94944)         0           maxpooling1d_24[0][0]            
____________________________________________________________________________________________________
dense_71 (Dense)                 (None, 32)            3038240     flatten_22[0][0]                 
___________________________________________________________________________________________